In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt
import warnings
warnings.filterwarnings('ignore')
import os
import sys

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from backtrade.utils import create_bt_data_feed, load_data_from_yahoo
from backtrade.strategy import CustomStrategy, VolumeBreakoutStrategy
from backtrade.utils import plot_performance_analysis, plot_backtest_results
from backtrade.utils import optimize_ma_strategy, run_backtest

plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)

In [4]:
df = load_data_from_yahoo("TSLA", "2020-01-01", "2023-12-31", save_to_csv=False)

获取 TSLA 的数据...


In [5]:
df.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02 00:00:00-05:00,28.299999,28.713333,28.114000,28.684000,142981500,0.0,0.0
2020-01-03 00:00:00-05:00,29.366667,30.266666,29.128000,29.534000,266677500,0.0,0.0
2020-01-06 00:00:00-05:00,29.364668,30.104000,29.333332,30.102667,151995000,0.0,0.0
2020-01-07 00:00:00-05:00,30.760000,31.441999,30.224001,31.270666,268231500,0.0,0.0
2020-01-08 00:00:00-05:00,31.580000,33.232666,31.215334,32.809334,467164500,0.0,0.0
2020-01-09 00:00:00-05:00,33.139999,33.253334,31.524668,32.089333,426606000,0.0,0.0
2020-01-10 00:00:00-05:00,32.119331,32.329334,31.580000,31.876667,194392500,0.0,0.0
2020-01-13 00:00:00-05:00,32.900002,35.042000,32.799999,34.990665,397764000,0.0,0.0
2020-01-14 00:00:00-05:00,36.284000,36.493999,34.993332,35.861332,434943000,0.0,0.0


In [4]:
# 执行回测
results, strategy = run_backtest(
    df=df, 
    strategy_class=VolumeBreakoutStrategy, 
    initial_cash=100000,
    commission=0.001
)

初始资金: 100000.00
2021-09-17: 买入信号: 价格=253.16, 数量=394, 交易量=84612600, 平均交易量=50344245, 止损=240.38, 止盈=278.74
2021-09-20: 买入执行: 价格=244.85, 数量=394, 成本=96472.21, 手续费=96.47
2021-09-20: 卖出信号(跟踪止损): 价格=243.39, 持仓数量=394
2021-09-21: 卖出执行: 价格=244.93, 数量=394, 收入=96472.21, 手续费=96.50
2021-09-21: 交易利润: 毛利=30.20, 净利=-162.77
2021-09-27: 买入信号: 价格=263.79, 数量=378, 交易量=84212100, 平均交易量=54802590, 止损=250.00, 止盈=291.35
2021-09-28: 买入执行: 价格=262.40, 数量=378, 成本=99187.20, 手续费=99.19
2021-10-04: 卖出信号(时间退出): 价格=260.51, 持仓数量=378
2021-10-05: 卖出执行: 价格=261.60, 数量=378, 收入=99187.20, 手续费=98.88
2021-10-05: 交易利润: 毛利=-302.40, 净利=-500.47
2021-10-21: 买入信号: 价格=298.00, 数量=333, 交易量=94444500, 平均交易量=59827890, 止损=283.94, 止盈=326.11
2021-10-22: 订单被拒绝或取消: 7
2021-10-25: 买入信号: 价格=341.62, 数量=290, 交易量=188556300, 平均交易量=65271270, 止损=322.21, 止盈=380.43
2021-10-26: 买入执行: 价格=341.56, 数量=290, 成本=99053.36, 手续费=99.05
2021-11-01: 卖出信号(时间退出): 价格=402.86, 持仓数量=290
2021-11-02: 卖出执行: 价格=386.45, 数量=290, 收入=99053.36, 手续费=112.07
2021-11-02: 交易利润: 毛利=13018.10, 净利=

In [5]:
bt_data = create_bt_data_feed(df)

# 定义参数优化范围
ma_short_range = (5, 20)   # 短期均线范围
ma_long_range = (20, 50)   # 长期均线范围
step = 5                   # 步长

# 执行参数优化
opt_results = optimize_ma_strategy(
    data=bt_data,
    ma_short_range=ma_short_range,
    ma_long_range=ma_long_range,
    step=step,
    commission=0.001,
    initial_cash=100000
)

# 显示优化结果
opt_results.head(10)

,short_ma,long_ma,roi,final_value,n_trades
0,5,50,0.253043,100253.042638,13
1,5,25,0.215828,100215.828066,21
2,5,35,0.149052,100149.052325,16
3,5,30,0.149014,100149.013668,21
4,10,20,0.132035,100132.035261,26
5,15,25,0.119400,100119.400125,19
6,10,25,0.097428,100097.428204,22
7,5,20,0.086372,100086.372052,28
8,5,40,0.059815,100059.814656,18
9,15,20,0.051125,100051.125265,28
